In [1]:
import requests
import json
import pandas as pd
from functools import reduce
from pandas.io.json import json_normalize

URL = 'http://35.186.149.233:8983/solr/dbnews/select'
T_URL = 'http://search:8983/solr/news/select'
def get_search_result(search_words, from_dt, to_dt):

    num_row = 100
    total = 1
    result_docs = []
    start = 0
    while total > start :
        params = {'debugQuery':'on','indent': 'on', 'q': 'title:"(' + search_words  + ')"~100 AND article:"' + search_words + '" AND datetime:[' + from_dt + 'T00:00:00Z TO ' + to_dt + 'T23:59:59Z]', 'sort': 'datetime desc', 'wt': 'json', 'start':start, 'rows':num_row}
        res = requests.get(URL, params=params)
        response = json.loads(res.text)
        total = response["response"]["numFound"]
        docs = response["response"]["docs"]
        #print("total:%d, start at %d" %(total, start))
        start = start + num_row
        result_docs.append(docs)

    merged_result = reduce(lambda x,y: x+y,result_docs)
    return merged_result


def get_tot_search_result(search_words, from_dt, to_dt):

    num_row = 100
    total = 1
    result_docs = []
    start = 0
    while total > start :
        params = {'debugQuery':'on','indent': 'on', 'q': 'title:"(' + search_words  + ')"~100 AND article:"' + search_words + '" AND datetime:[' + from_dt + 'T00:00:00Z TO ' + to_dt + 'T23:59:59Z]', 'sort': 'datetime desc', 'wt': 'json', 'start':start, 'rows':num_row}
        res = requests.get(T_URL, params=params)
        response = json.loads(res.text)
        total = response["response"]["numFound"]
        docs = response["response"]["docs"]
        #print("total:%d, start at %d" %(total, start))
        start = start + num_row
        result_docs.append(docs)

    merged_result = reduce(lambda x,y: x+y,result_docs)
    return merged_result

In [88]:
week1 = ['2017-03-27','2017-04-02']
week2 = ['2017-04-03','2017-04-09']
week3 = ['2017-04-10','2017-04-16']
week4 = ['2017-04-17','2017-04-23']
week5 = ['2017-04-24','2017-04-30']

week = [
    ['2017-03-27','2017-04-02'], 
    ['2017-04-03','2017-04-09'],
    ['2017-04-10','2017-04-16'],
    ['2017-04-17','2017-04-23'],
    ['2017-04-24','2017-04-30']   
       ]

In [92]:
for weekly in week:    
    df_tot_moon_result = json_normalize(get_tot_search_result('문재인', weekly[0], weekly[1]))
    df_tot_moon_result['during'] = weekly
    df_tot_ahn_result = json_normalize(get_tot_search_result('안철수', weekly[0], weekly[1]))
    df_tot_ahn_result['during'] = weekly
    df_tot_hong_result = json_normalize(get_tot_search_result('홍준표', weekly[0], weekly[1]))
    df_tot_hong_result['during'] = weekly
    df_tot_you_result = json_normalize(get_tot_search_result('유승민', weekly[0], weekly[1]))
    df_tot_you_result['during'] = weekly
    df_tot_sim_result = json_normalize(get_tot_search_result('심상정', weekly[0], weekly[1]))
    df_tot_sim_result['during'] = weekly
    
    df_moon_result = json_normalize(get_search_result('문재인', weekly[0], weekly[1]))
    df_moon_result['during'] = weekly
    df_ahn_result = json_normalize(get_search_result('안철수',  weekly[0], weekly[1]))
    df_ahn_result['during'] = weekly
    df_hong_result = json_normalize(get_search_result('홍준표', weekly[0], weekly[1]))
    df_hong_result['during'] = weekly
    df_you_result = json_normalize(get_search_result('유승민',  weekly[0], weekly[1]))
    df_you_result['during'] = weekly
    df_sim_result = json_normalize(get_search_result('심상정',  weekly[0], weekly[1]))
    df_sim_result['during'] = weekly

ValueError: Length of values does not match length of index

In [90]:
df_moon_result['tot'] = df_moon_result['n_shares'].fillna(0) + df_moon_result['o_shares'].fillna(0) 
df_ahn_result['tot'] = df_ahn_result['n_shares'].fillna(0) + df_ahn_result['o_shares'].fillna(0) 
df_hong_result['tot'] = df_hong_result['n_shares'].fillna(0) + df_hong_result['o_shares'].fillna(0) 
df_you_result['tot'] = df_you_result['n_shares'].fillna(0) + df_you_result['o_shares'].fillna(0) 
df_sim_result['tot'] = df_sim_result['n_shares'].fillna(0) + df_sim_result['o_shares'].fillna(0) 

In [72]:
tot = len(df_moon_result) + len(df_ahn_result) + len(df_hong_result) + len(df_you_result)+ len(df_sim_result)

In [73]:
tot_article = len(df_tot_moon_result) + len(df_tot_ahn_result) + len(df_tot_hong_result) + len(df_tot_you_result)+ len(df_tot_sim_result)

# 후보자별 공유된 뉴스 비율

후보자별 공유된 뉴스 갯수/전체 공유된 뉴스 합

In [74]:
print("문재인 : %0.2f" %(len(df_moon_result)*100/tot))
print("안철수 : %0.2f" %(len(df_ahn_result)*100/tot))
print("홍준표 : %0.2f" %(len(df_hong_result)*100/tot))
print("유승민 : %0.2f" %(len(df_you_result)*100/tot))
print("심상정 : %0.2f" %(len(df_sim_result)*100/tot))

문재인 : 33.25
안철수 : 27.55
홍준표 : 19.71
유승민 : 11.16
심상정 : 8.31


In [75]:
print("문재인 : %0.2f" %(len(df_tot_moon_result)*100/tot_article))
print("안철수 : %0.2f" %(len(df_tot_ahn_result)*100/tot_article))
print("홍준표 : %0.2f" %(len(df_tot_hong_result)*100/tot_article))
print("유승민 : %0.2f" %(len(df_tot_you_result)*100/tot_article))
print("심상정 : %0.2f" %(len(df_tot_sim_result)*100/tot_article))

문재인 : 35.69
안철수 : 34.46
홍준표 : 14.17
유승민 : 9.46
심상정 : 6.22


In [76]:
shared_tot = df_moon_result['tot'].sum()+df_ahn_result['tot'].sum()+df_you_result['tot'].sum()+df_hong_result['tot'].sum()+df_sim_result['tot'].sum()

# 후보자별 뉴스 공유수 비율

후보자별 뉴스 공유수 합/전체 뉴스 공유 수

In [77]:
print("문재인 : %0.2f" %(df_moon_result['tot'].sum()*100/shared_tot))
print("안철수 : %0.2f" %(df_ahn_result['tot'].sum()*100/shared_tot))
print("홍준표 : %0.2f" %(df_hong_result['tot'].sum()*100/shared_tot))
print("유승민 : %0.2f" %(df_you_result['tot'].sum()*100/shared_tot))
print("심상정 : %0.2f" %(df_sim_result['tot'].sum()*100/shared_tot))

문재인 : 31.58
안철수 : 27.26
홍준표 : 17.79
유승민 : 11.26
심상정 : 12.11


In [78]:
moon_by_media = df_moon_result[['ns', 'n_shares', 'o_shares', 'tot']].groupby('ns')
ahn_by_media = df_ahn_result[['ns', 'n_shares', 'o_shares', 'tot']].groupby('ns')
hong_by_media = df_hong_result[['ns', 'n_shares', 'o_shares', 'tot']].groupby('ns')
you_by_media = df_you_result[['ns', 'n_shares', 'o_shares', 'tot']].groupby('ns')
sim_by_media = df_sim_result[['ns', 'n_shares', 'o_shares', 'tot']].groupby('ns')

In [79]:
tot_moon_by_media = df_tot_moon_result[['oid', 'id']].groupby('oid')
tot_ahn_by_media = df_tot_ahn_result[['oid', 'id']].groupby('oid')
tot_hong_by_media = df_tot_hong_result[['oid', 'id']].groupby('oid')
tot_you_by_media = df_tot_you_result[['oid', 'id']].groupby('oid')
tot_sim_by_media = df_tot_sim_result[['oid', 'id']].groupby('oid')

# 문재인 후보 언론사별 공유수

In [80]:
moon_by_media.sum()

,n_shares,o_shares,tot
ns,,,
news2016.chosun,2.0,7.0,9.0
news2016.donga,5.0,33.0,38.0
news2016.edaily,38.0,9.0,47.0
news2016.hani,NaN,1.0,1.0
news2016.joins,17.0,53.0,70.0
news2016.jtbc,2.0,8.0,10.0
news2016.kbs,10.0,11.0,21.0
news2016.kh,1.0,11.0,12.0
news2016.mbc,NaN,2.0,2.0


In [59]:
tot_moon_by_media.count()

,id
oid,
001,42
008,220
009,48
018,117
020,73
023,21
025,81
028,33
032,67


# 안철수 후보 언론사별 공유수

In [81]:
ahn_by_media.sum()

,n_shares,o_shares,tot
ns,,,
news2016.chosun,3.0,8.0,11.0
news2016.donga,1.0,12.0,13.0
news2016.edaily,20.0,3.0,23.0
news2016.hani,2.0,5.0,7.0
news2016.joins,6.0,5.0,11.0
news2016.jtbc,6.0,6.0,12.0
news2016.kbs,21.0,NaN,21.0
news2016.kh,6.0,11.0,17.0
news2016.mbc,NaN,3.0,3.0


In [61]:
tot_ahn_by_media.count()

,id
oid,
001,39
008,219
009,33
018,69
020,42
023,15
025,54
028,25
032,54


# 홍준표 후보 언론사별 공유수

In [82]:
hong_by_media.sum()

,n_shares,o_shares,tot
ns,,,
news2016.chosun,NaN,11.0,11.0
news2016.donga,3.0,5.0,8.0
news2016.edaily,31.0,1.0,32.0
news2016.hani,NaN,1.0,1.0
news2016.joins,4.0,11.0,15.0
news2016.jtbc,NaN,5.0,5.0
news2016.kbs,1.0,13.0,14.0
news2016.kh,3.0,13.0,16.0
news2016.mk,NaN,1.0,1.0


# 유승민 후보 언론사별 공유수

In [83]:
you_by_media.sum()

,n_shares,o_shares,tot
ns,,,
news2016.chosun,NaN,7.0,7.0
news2016.donga,3.0,5.0,8.0
news2016.edaily,5.0,NaN,5.0
news2016.joins,7.0,3.0,10.0
news2016.jtbc,NaN,1.0,1.0
news2016.kh,3.0,18.0,21.0
news2016.mk,1.0,2.0,3.0
news2016.mt,10.0,2.0,12.0
news2016.nocut,1.0,NaN,1.0


# 심상정 후보 언론사별 공유수

In [84]:
sim_by_media.sum()

,n_shares,o_shares,tot
ns,,,
news2016.donga,1.0,13.0,14.0
news2016.edaily,9.0,NaN,9.0
news2016.hani,NaN,1.0,1.0
news2016.joins,NaN,1.0,1.0
news2016.jtbc,1.0,2.0,3.0
news2016.kh,1.0,6.0,7.0
news2016.mt,9.0,1.0,10.0
news2016.nocut,2.0,44.0,46.0
news2016.ohmynews,NaN,1.0,1.0


In [86]:
df_sim_result[df_sim_result["ns"]=='news2016.nocut']

,_ts,_version_,author,datetime,id,n_shares,naver_url,ns,o_shares,org_url,section,title,tot
5,6412305822679826434,1565504423989346304,CBS노컷뉴스 조태임 기자,2017-04-23T00:00:00Z,58fd162c977b5cbef9162c12,NaN,http://news.naver.com/main/read.nhn?mode=LSD&m...,news2016.nocut,44.0,http://www.nocutnews.co.kr/news/4773193,정치,역풍 우려했나…文 공격 줄인 심상정,44.0
34,6409980982652305410,1564936826448248832,CBS 스마트뉴스팀 김세준 · 김기현 기자,2017-04-17T00:00:00Z,58f4d3be977b5cbef9160bbf,2.0,http://news.naver.com/main/read.nhn?mode=LSD&m...,news2016.nocut,NaN,http://www.nocutnews.co.kr/news/4769397,정치,"[영상] 심상정, 노동자 피땀 서린 구로공단서 출정식",2.0
